sets up gdrive


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount=True)
root_dir = "/content/gdrive/My Drive/"
#base_dir = root_dir + 'fastai-v3/'

Mounted at /content/gdrive


the following three lines ensure that any edits to libraries you make are reloaded here automatically, and also that any charts or images displayed are shown in this notebook.

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

 install secure connection library

In [3]:
!pip install 'pyOpenSSL>=0.11'

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 3.2MB 5.9MB/s 


In [4]:
!pip install earthengine-api 

authenticate earthengine to work with your google account

In [5]:
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=uDHpF_KRdDJIGLwvkKycUw9kMV1UjK8a4ca4gETUeUs&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g433T9ry_PWUVFbZAK8E5k84D3oVsP1mDqa4X-IZrcNy_BywvgFGBo

Successfully saved authorization token.


test correct earthengine setup for notebook, if no error then ok :)

In [6]:
import ee; ee.Initialize();

In [7]:
!pip install -U -q PyDrive

sets up gdrive within earthengine for input and output

In [8]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import csv from gdrive

In [12]:
#paste into link the link of csv file on your gdrive
import pandas as pd
# Cambiar el id, "Obtener link"
link = 'https://drive.google.com/open?id=1KhD90vGEdvV7qwSt1W6TQnQYAs_PUdkP'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('CNN_CENI_COL_79979_ALL.csv')  
df3 = pd.read_csv('CNN_CENI_COL_79979_ALL.csv')

#cnn_col_2018_annual_month_median_nl

1KhD90vGEdvV7qwSt1W6TQnQYAs_PUdkP


check sample of csv

In [10]:
df3.shape

(77979, 3)

In [13]:
# Set the id = rownumber as index of the DataFrame
df3 = df3.set_index('id')

df3 = df3.sort_values(by=['id'])
df3.head()

,lat,long
id,,
1,-4.191180,-69.973730
2,-4.191180,-69.939235
3,-4.156684,-69.973730
4,-4.156684,-69.939235
5,-4.122189,-70.008225


In [15]:
#SENTINEL export

import ee

ee.Initialize()

##################################
#manually created bounding polygon for Colombia

#COLOMBIA
tha_shp = ee.Geometry.Polygon(
        [[[-78.9909352282, 12.4373031682],
          [-78.9909352282, -4.29818694419],
          [-66.8763258531, -4.29818694419],
          [-66.8763258531, 12.4373031682]]]);

#SENTINEL START
#/**
# * Function to mask clouds using the Sentinel-2 QA band
# * @param {ee.Image} image Sentinel-2 image
# * @return {ee.Image} cloud masked Sentinel-2 image
# */
def maskS2clouds(image):
    qa = image.select('QA60')
#  // Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
#  // Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)


#// Map the function over one year of data and take the median.
#// Load Sentinel-2 TOA reflectance data.

rgbVis = {'min': 0.0,'max': 0.3,'bands': ['B4', 'B3', 'B2'],}

dataset = ee.ImageCollection('COPERNICUS/S2').filterDate('2016-01-01', '2016-12-31').filterBounds(tha_shp).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)).map(maskS2clouds)

upres = dataset.median().visualize(**rgbVis)

#################################

#downloaded image is black -> switch coordinates :)

#all bands must have the same data type when exporting, you should download only the bands you need and cast them to a type that suit all bands.
##################################

# gives a tuple of column name and series 
# for each column in the dataframe 

#i = 1
#77979
for i in range(1, 77979):
  if i > 0 and i < 20 :
    c_lon = df3['long'][i]
    c_lat = df3['lat'][i]
    geometry = ee.Geometry.Point([c_lon, c_lat]).buffer(1920)#1920metres because we want resolution of image 384x384 pixels -> 10x384/2 = 1920
    geometry = geometry.getInfo()['coordinates'][0]
    task_config = {
        'scale': 10,
        'region': geometry,
        'driveFolder': 'CNN_IMG_COL_2016_3840_TIF_79979_ALL_0_20k',
    }

    task = ee.batch.Export.image(upres, 'CNN_DIMG_COL_2018_ST_384_3840_13_' + '{:06d}'.format(i), task_config)
  
        #print('Export Image '+ str(i)+ ' was submitted, please wait ...');

    task.start()
